# DS-SF-34 | 18 | Natural Language Processing | Codelong | Starter Code

## >>> One-time setup

In [2]:

import nltk
nltk.download()

pass

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


## <<< One-time setup

## Part A | Tokenization and Stemming

In [1]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import string
import unicodedata
from nltk import tokenize, corpus, stem

from sklearn import feature_extraction, linear_model, ensemble, model_selection, metrics, decomposition

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

## Tokenization

In [4]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [5]:
def tokenize_text(document):
    document = document.encode('utf-8')

    # Convert text to lowercase
    document = document.lower()

    # Tokenize
    tokens = tokenize.word_tokenize(document)

    # Remove punctuation in tokens and then remove empty tokens
    tokens = [token.translate(None, string.punctuation) for token in tokens]
    tokens = [token for token in tokens if token]

    # Remove stop words
    tokens = [token for token in tokens if not token in corpus.stopwords.words('english')]

    return tokens

In [6]:
tokens = tokenize_text("This is a sentence...  Wait, here's another.  And a third!")

tokens

TypeError: cannot use a string pattern on a bytes-like object

## Stemming

In [ ]:
class Stemmer:
    stemmer = stem.porter.PorterStemmer()

    @staticmethod
    def stem_tokens(tokens):
        return [Stemmer.stemmer.stem(token) for token in tokens]

In [ ]:
tokens = Stemmer.stem_tokens(tokens)

tokens

## Part B | Book reviews

Below, we will be analyzing a partial list of the reviews for J.K. Rowling's The Casual Vacancy.  (https://www.amazon.com/dp/0316228532)  We scrapped this dataset during class 3.

In [2]:
df = pd.read_csv(os.path.join('..', 'datasets', 'dataset-18-reviews.csv'))

In [3]:
df

,date,id,author,title,body,star_rating
0,2017-04-21,R3TUANQ2EB3ECB,MichaelMichaels,Skip it. Life is too short.,I've never read any of the Harry Potter books ...,1.0
1,2017-04-20,R2DD03ZZ4218VW,Frans van Wyk,Four Stars,Excellent Read with a lot of real life values ...,4.0
2,2017-04-20,R296NVKLH5QS4W,Sabina Duke,Characters,Hard to keep the characters straight,4.0
3,2017-04-05,R3MP7W8LH6VHU8,Jen Blau,GIVE IT A CHANCE!,I almost put this book down. I'm new to Rowlin...,5.0
4,2017-04-04,RZWP48RKJCXT1,Lilith Eleanor,Frighteningly good,Amazing. Rowling combines fantastic writing wi...,5.0
...,...,...,...,...,...,...
5856,2012-09-27,RT2TE0W92SL67,Tricia K.,Seriously? $17 bucks for a computer file??? ...,Premise sounds dull as dirt. For $17 for a co...,1.0
5857,2012-09-27,R14ZGYPSP9H0Y7,Pretzel,A must read,The depth of character development and storyli...,5.0
5858,2012-09-27,R1913ISIDAGQ1A,Prodigy,I love it,The book was great and I will love to re-read ...,5.0
5859,2012-09-27,R2JY771IW7RI3R,David Katz,Kendle price too expensive,I started to order the kindle edition and than...,5.0


In [4]:
df.drop(['date', 'id', 'author', 'title'],
    axis = 1,
    inplace = True)

In [5]:
df

,body,star_rating
0,I've never read any of the Harry Potter books ...,1.0
1,Excellent Read with a lot of real life values ...,4.0
2,Hard to keep the characters straight,4.0
3,I almost put this book down. I'm new to Rowlin...,5.0
4,Amazing. Rowling combines fantastic writing wi...,5.0
...,...,...
5856,Premise sounds dull as dirt. For $17 for a co...,1.0
5857,The depth of character development and storyli...,5.0
5858,The book was great and I will love to re-read ...,5.0
5859,I started to order the kindle edition and than...,5.0


### `NaN`

In [9]:
# TODO
df.isnull().sum()

body           0
star_rating    0
dtype: int64

In [8]:
df.dropna(inplace = True)

### Positive, neutral, and negatives reviews

In [10]:
# TODO
df['polarity'] = df.star_rating.map({1: -1, 2: -1, 3: 0, 4: 1, 5: 1})

In [16]:
df.polarity.value_counts()

 1    2711
-1    2177
 0     970
Name: polarity, dtype: int64

In [17]:
ns = df.polarity.value_counts()

In [19]:
ns.min()

970

In [20]:
for polarity in [-1, 0, 1]:
    n = ns[polarity] - ns.min()
    index = df[df.polarity == polarity].sample(n = n, random_state = 0).index
    df.drop(index, inplace = True)

In [22]:
df.polarity.value_counts()

 1    970
-1    970
 0    970
Name: polarity, dtype: int64

### Feature matrix and response vector

In [23]:
# TODO
X = df.body
c = df.polarity

### Train/test sets

In [24]:
train_X, test_X, train_c, test_c = model_selection.train_test_split(X, c, stratify = c, train_size = .6, random_state = 0)

In [27]:
test_c.value_counts()

-1    388
 1    388
 0    388
Name: polarity, dtype: int64

### TF-IDF and `TfidfVectorizer`

In [28]:
# TODO

vectorizer = feature_extraction.text.TfidfVectorizer(stop_words = 'english')

### Bag-of-words

In [ ]:
vectorizer.get_feature_names()

### Transformed feature matrix `X`

In [29]:
# TODO
class CustomTokenizer(object):
    def __call__(self, document):
        tokens = tokenize_text(document)
        tokens = Stemmer.stem_tokens(tokens)
        return tokens

In [30]:
vectorizer = feature_extraction.text.TfidfVectorizer(tokenizer = CustomTokenizer())

In [31]:
vectorizer.fit(train_X)

NameError: name 'tokenize_text' is not defined

### Machine Learning Modeling

> # TODO...